<a href="https://colab.research.google.com/github/AdityaDurgapal/from_Scratch/blob/main/GAN_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.layers import Reshape
import matplotlib.pyplot as plt

In [34]:
def discriminator(inputshape=(32,32,3)):
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same', input_shape=inputshape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(64, (3, 3), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(256, (3, 3), strides=(2, 2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation='sigmoid'))
  opt =  Adam(learning_rate=0.0002,beta_1=0.5)
  model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
  return model

In [35]:
model= discriminator()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_30 (LeakyReLU)      │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_31 (LeakyReLU)      │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_32 (LeakyReLU)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_33 (LeakyReLU)      │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 392,513 (1.50 MB)

 Trainable params: 392,513 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
def generate_real_samples(dataset,n_samples):
  idx=np.random.randint(0,dataset.shape[0],n_samples)
  X=dataset[idx]
  y=np.ones((n_samples,1))
  return X,y

In [37]:
def generate_fake_samples(g_model,latent_dim, n_samples):
  x_input=generate_latent_points(latent_dim,n_samples)
  X=g_model.predict(x_input)
  y=np.zeros((n_samples,1))
  return X,y

In [38]:
def train_discriminator(model,dataset,n_iter=100,n_batch=128):
  half_batch=int(n_batch/2)
  for i in range(n_iter):
    X_real,y_real=generate_real_samples(dataset,half_batch)
    real_acc=model.train_on_batch(X_real,y_real)
    X_fake,y_fake=generate_fake_samples(g_model,latent_dim,half_batch)
    fake_acc=model.train_on_batch(X_fake,y_fake)
    print('>%d real=%.0f%% fake=%.0f%%'%(i+1,real_acc[1]*100,fake_acc[1]*100))

In [39]:
def generator(latent_dim):
  model=Sequential()
  n_nodes=256*4*4
  model.add(Dense(n_nodes,input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((4,4,256)))
  model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2DTranspose(128,(4,4),strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Conv2D(3,(3,3),activation='tanh',padding='same'))
  return model

In [40]:
latent_dim=100
model=generator(latent_dim)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 4096)           │       413,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_34 (LeakyReLU)      │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_9              │ (None, 8, 8, 128)      │       524,416 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_35 (LeakyReLU)      │ (None, 8, 8, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_10             │ (None, 16, 16, 128)    │       262,272 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_36 (LeakyReLU)      │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_11             │ (None, 32, 32, 128)    │       262,272 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_37 (LeakyReLU)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 32, 32, 3)      │         3,459 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,466,115 (5.59 MB)

 Trainable params: 1,466,115 (5.59 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
def generate_latent_points(latent_dim,n_samples):
  x_input=np.random.randn(latent_dim*n_samples)
  x_input=x_input.reshape((n_samples,latent_dim))
  return x_input

In [42]:
def gan(g_model,d_model):
  d_model.trainable=False
  model=Sequential()
  model.add(g_model)
  model.add(d_model)
  opt=Adam(learning_rate=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy',optimizer=opt)
  return model

In [43]:
latent_dim=100
d_model=discriminator()
g_model=generator(latent_dim)
gan_model=gan(g_model,d_model)
gan_model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_14 (Sequential)      │ (None, 32, 32, 3)      │     1,466,115 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_13 (Sequential)      │ (None, 1)              │       392,513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,858,628 (7.09 MB)

 Trainable params: 1,466,115 (5.59 MB)

 Non-trainable params: 392,513 (1.50 MB)

In [44]:
def train(g_model,d_model,gan_model,dataset,latent_dim,n_epochs=100,n_batch=128):
  bat_per_epo=int(dataset.shape[0]/n_batch)
  half_batch=int(n_batch/2)
  for i in range(n_epochs):
    for j in range(bat_per_epo):
      X_real,y_real=generate_real_samples(dataset,half_batch)
      d_loss1,_=d_model.train_on_batch(X_real,y_real)
      X_fake,y_fake=generate_fake_samples(g_model,latent_dim,half_batch)
      d_loss2,_=d_model.train_on_batch(X_fake,y_fake)
      X_gan=generate_latent_points(latent_dim,n_batch)
      y_gan=np.ones((n_batch,1))
      g_loss=gan_model.train_on_batch(X_gan,y_gan)
      print('>%d,%d/%d,d1=%.3f,d2=%.3f,g=%.3f'%(i+1,j+1,bat_per_epo,d_loss1,d_loss2,g_loss))

    if (i+1)%10==0:
      summarize_performance(i,g_model,d_model,dataset,latent_dim)


In [45]:
def summarize_performance(epoch,g_model,d_model,dataset,latent_dim,n_samples=100):
  X_real,y_real=generate_real_samples(dataset,n_samples)
  _,acc_real=d_model.evaluate(X_real,y_real,verbose=0)
  X_fake,y_fake=generate_fake_samples(g_model,latent_dim,n_samples)
  _,acc_fake=d_model.evaluate(X_fake,y_fake,verbose=0)
  print("Accuracy real: %.0f%%, fake: %.0f%%" % (acc_real*100, acc_fake*100))
  save_plot(X_fake,epoch)
  filename = 'generator_model_%03d.h5' % (epoch + 1)
  g_model.save(filename)

In [46]:
def save_plot(examples,epoch,n=10):
  examples = (examples + 1) / 2.0
  for i in range(n*n):
    plt.subplot(n,n,i+1)
    plt.axis('off')
    plt.imshow(examples[i,:,:,0],cmap='gray_r')
    filename = 'generated_plot_e%03d.png' % (epoch+1)
  plt.savefig(filename)
  plt.close()


In [47]:
def load_real_samples():
    from tensorflow.keras.datasets import cifar10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = (x_train.astype('float32') - 127.5) / 127.5
    return x_train

In [ ]:
dataset = load_real_samples()
latent_dim = 100
d_model = discriminator()
g_model = generator(latent_dim)
gan_model = gan(g_model, d_model)
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=200, n_batch=128)

/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:83: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 243ms/step
>1,1/390,d1=0.695,d2=0.694,g=0.693
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step
>1,2/390,d1=0.694,d2=0.694,g=0.693
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
>1,3/390,d1=0.694,d2=0.694,g=0.692
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
>1,4/390,d1=0.694,d2=0.694,g=0.692
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
>1,5/390,d1=0.694,d2=0.694,g=0.691
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 370ms/step
>1,6/390,d1=0.695,d2=0.696,g=0.690
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step
>1,7/390,d1=0.696,d2=0.696,g=0.689
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step
>1,8/390,d1=0.696,d2=0.697,g=0.687
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
>1,9/390,d1=0.697,d2=0.698,g=0.684
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 239ms/step
>1,10/390,d1=0.698,d2=0.700,g=0.681
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 241ms/step
>1,11/390,d1=0.700,d2=0.702,g=0.677
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
>1,12/390,d1=0.702,d2=0.705,g=0.672
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 390ms/step
>1,13/390,d1=0.704,d2=0.708,g=0.666
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/